In [1]:
import json
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from itertools import islice
from datetime import datetime as dt
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
#data = open('data/AllPrintings.json', 'r', encoding='utf-8')
#data = json.load(data)
#prices = open('data/AllPrices.json', 'r', encoding='utf-8')
#prices = json.load(data/prices)
#prices = prices['data']

In [3]:
def getCardData(card, prices):
    text = power = toughness = manaCost = isReprint = ''
    if 'text' in card:
        text = card['text']
    if 'power' in card:
        power = card['power']
    if 'toughness' in card:
        toughness = card['toughness']
    if 'manaCost' in card:
        manaCost = card['manaCost'].replace('}{',' ').replace('{','').replace('}','')
    if 'isReprint' in card:
        isReprint = card['isReprint']
    card = {'name' : card['name'],
            'manaCost' : manaCost,
            'type' : card['type'],
            'text' : text,
            'power' : power,
            'toughness' : toughness,
            'setCode' : card['setCode'],
            'rarity' : card['rarity'],
            'legalities' : card['legalities'],
            'isReprint' : isReprint,
            'prices': prices}
    return card

def prepareCards(data, adjustedPrices):
    cards = {}
    for k, sets in data['data'].items():
        for card in sets['cards']:
            if card['uuid'] not in adjustedPrices.keys(): continue
            cards[card['uuid']] = getCardData(card, adjustedPrices[card['uuid']])
    return cards

def adjustPriceData(filteredPrices):
    ajustedPrices = {}
    for uuid, provider in filteredPrices.items():
        if 'cardkingdom' in provider.keys():
            if 'foil' in provider['cardkingdom'].keys():
                ckf = provider['cardkingdom']['foil']
            else:
                ckf = 0
        if 'cardkingdom' in provider.keys():
            if 'normal' in provider['cardkingdom'].keys():
                ckn = provider['cardkingdom']['normal']
            else:
                ckn = 0
        if 'tcgplayer' in provider.keys():
            if 'foil' in provider['tcgplayer'].keys():
                tcgf = provider['tcgplayer']['foil']
            else:
                tcgf = 0
        if 'tcgplayer' in provider.keys():
            if 'normal' in provider['tcgplayer'].keys():
                tcgn = provider['tcgplayer']['normal']
            else:
                tcgn = 0
        ajustedPrices[uuid] = [ckf, ckn, tcgf, tcgn]
    return ajustedPrices

def getPrices(prices):
    filteredPrices = {}
    for uuid, form in prices.items():
        for typeofcard, lists in form.items():
            prov = {}
            for provider, buylist in lists.items():
                types = {}
                if 'buylist' in buylist.keys():
                    for quality, dates in buylist['buylist'].items():
                        date = max(dates.keys(), key=lambda d: dt.strptime(d, '%Y-%m-%d'))
                        types[quality] = dates[date]
                prov[provider] = types
        filteredPrices[uuid] = prov
    filteredPrices = adjustPriceData(filteredPrices)
    return filteredPrices

In [4]:
#adjustedPrices = getPrices(prices)
#cards = prepareCards(data, adjustedPrices)

In [5]:
#print(dict(islice(adjustedPrices.items(), 0, 10)))

In [6]:
#data = pd.DataFrame.from_dict(cards, orient='index').reset_index(drop=True)
#data.to_pickle('data/data.pkl')
data = pd.read_pickle('data/data.pkl')

In [7]:
y = data['prices'].tolist()

In [8]:
x = data.drop(columns=['prices'])
x.size

664670

In [9]:
len(y)

66467

In [10]:
x.head()

,name,manaCost,type,text,power,toughness,setCode,rarity,legalities,isReprint
0,Ancestor's Chosen,5 W W,Creature — Human Cleric,First strike (This creature deals combat damag...,4,4,10E,uncommon,"{'commander': 'Legal', 'duel': 'Legal', 'legac...",True
1,Ancestor's Chosen,5 W W,Creature — Human Cleric,First strike (This creature deals combat damag...,4,4,10E,uncommon,"{'commander': 'Legal', 'duel': 'Legal', 'legac...",True
2,Angel of Mercy,4 W,Creature — Angel,Flying\nWhen Angel of Mercy enters the battlef...,3,3,10E,uncommon,"{'commander': 'Legal', 'duel': 'Legal', 'gladi...",True
3,Angel of Mercy,4 W,Creature — Angel,Flying\nWhen Angel of Mercy enters the battlef...,3,3,10E,uncommon,"{'commander': 'Legal', 'duel': 'Legal', 'gladi...",True
4,Angelic Blessing,2 W,Sorcery,Target creature gets +3/+3 and gains flying un...,,,10E,common,"{'commander': 'Legal', 'duel': 'Legal', 'legac...",True


In [11]:
data.head()

,name,manaCost,type,text,power,toughness,setCode,rarity,legalities,isReprint,prices
0,Ancestor's Chosen,5 W W,Creature — Human Cleric,First strike (This creature deals combat damag...,4,4,10E,uncommon,"{'commander': 'Legal', 'duel': 'Legal', 'legac...",True,"[0, 0.26, 0, 0.58]"
1,Ancestor's Chosen,5 W W,Creature — Human Cleric,First strike (This creature deals combat damag...,4,4,10E,uncommon,"{'commander': 'Legal', 'duel': 'Legal', 'legac...",True,"[0.08, 0, 0.02, 0.01]"
2,Angel of Mercy,4 W,Creature — Angel,Flying\nWhen Angel of Mercy enters the battlef...,3,3,10E,uncommon,"{'commander': 'Legal', 'duel': 'Legal', 'gladi...",True,"[1.58, 0.1, 4.45, 0.01]"
3,Angel of Mercy,4 W,Creature — Angel,Flying\nWhen Angel of Mercy enters the battlef...,3,3,10E,uncommon,"{'commander': 'Legal', 'duel': 'Legal', 'gladi...",True,"[0.6, 0, 0.5, 0.01]"
4,Angelic Blessing,2 W,Sorcery,Target creature gets +3/+3 and gains flying un...,,,10E,common,"{'commander': 'Legal', 'duel': 'Legal', 'legac...",True,"[0.12, 0.05, 0.35, 0]"


In [12]:
# Process the input data
x['text'] = x[x.columns[1:]].apply(lambda x: ' '.join(x.dropna().astype(str)),axis=1)
x = x['text'].tolist()
x = np.array(x)
y = np.array(y)
VOCAB_SIZE = 15000

In [13]:
# Define the encoder
encoder = tf.keras.layers.TextVectorization(max_tokens=VOCAB_SIZE)
encoder.adapt(x)

# Define the model
model = tf.keras.Sequential([
    encoder,  # Add the encoder to the model
    tf.keras.layers.Embedding(
        input_dim=VOCAB_SIZE,
        output_dim=128,
        # Use masking to handle the variable sequence lengths
        mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences=True)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128)),
    tf.keras.layers.Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(4)
])
# Compile the model with an optimizer and a loss function
model.compile(optimizer='adam', loss='mean_absolute_error')

In [14]:
# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(x, y, test_size=0.2)
X_val, X_test, y_val, y_test = train_test_split(x, y, test_size=0.5)

In [15]:
# Train the model
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
plateau = tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.1, patience=10, verbose=0, mode="auto", min_delta=0.0001, cooldown=0, min_lr=0)
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath= 'model/checkpoint.ckpt', save_weights_only=True, verbose=1)


history = model.fit(X_train, y_train,
                    epochs=250,
                    validation_data=(X_val, y_val),
                    batch_size=64,
                    callbacks=[early_stopping, plateau, checkpoint])

Epoch 1/250
830/831 [============================>.] - ETA: 0s - loss: 5.9587
Epoch 1: saving model to model\checkpoint.ckpt
831/831 [==============================] - 66s 66ms/step - loss: 5.9543 - val_loss: 5.5913 - lr: 0.0010
Epoch 2/250
830/831 [============================>.] - ETA: 0s - loss: 5.7480
Epoch 2: saving model to model\checkpoint.ckpt
831/831 [==============================] - 51s 61ms/step - loss: 5.7437 - val_loss: 5.3743 - lr: 0.0010
Epoch 3/250
830/831 [============================>.] - ETA: 0s - loss: 5.4682
Epoch 3: saving model to model\checkpoint.ckpt
831/831 [==============================] - 50s 61ms/step - loss: 5.4651 - val_loss: 5.0659 - lr: 0.0010
Epoch 4/250
830/831 [============================>.] - ETA: 0s - loss: 5.3526
Epoch 4: saving model to model\checkpoint.ckpt
831/831 [==============================] - 51s 61ms/step - loss: 5.3584 - val_loss: 5.0058 - lr: 0.0010
Epoch 5/250
830/831 [============================>.] - ETA: 0s - loss: 5.1626
Epoch 

In [16]:
# Evaluate the model on the test set
test_loss= model.evaluate(X_test, y_test, return_dict=True)
print('Test loss:', test_loss)

1039/1039 [==============================] - 18s 17ms/step - loss: 2.6464
Test loss: {'loss': 2.646355390548706}


In [24]:
model.save('model/model', save_format="tf")

INFO:tensorflow:Assets written to: model/model\assets


INFO:tensorflow:Assets written to: model/model\assets


In [21]:
data.head(50)

,name,manaCost,type,text,power,toughness,setCode,rarity,legalities,isReprint,prices
0,Ancestor's Chosen,5 W W,Creature — Human Cleric,First strike (This creature deals combat damag...,4,4,10E,uncommon,"{'commander': 'Legal', 'duel': 'Legal', 'legac...",True,"[0, 0.26, 0, 0.58]"
1,Ancestor's Chosen,5 W W,Creature — Human Cleric,First strike (This creature deals combat damag...,4,4,10E,uncommon,"{'commander': 'Legal', 'duel': 'Legal', 'legac...",True,"[0.08, 0, 0.02, 0.01]"
2,Angel of Mercy,4 W,Creature — Angel,Flying\nWhen Angel of Mercy enters the battlef...,3,3,10E,uncommon,"{'commander': 'Legal', 'duel': 'Legal', 'gladi...",True,"[1.58, 0.1, 4.45, 0.01]"
3,Angel of Mercy,4 W,Creature — Angel,Flying\nWhen Angel of Mercy enters the battlef...,3,3,10E,uncommon,"{'commander': 'Legal', 'duel': 'Legal', 'gladi...",True,"[0.6, 0, 0.5, 0.01]"
4,Angelic Blessing,2 W,Sorcery,Target creature gets +3/+3 and gains flying un...,,,10E,common,"{'commander': 'Legal', 'duel': 'Legal', 'legac...",True,"[0.12, 0.05, 0.35, 0]"
5,Angelic Blessing,2 W,Sorcery,Target creature gets +3/+3 and gains flying un...,,,10E,common,"{'commander': 'Legal', 'duel': 'Legal', 'legac...",True,"[0.08, 0, 0.01, 0.01]"
6,Angelic Chorus,3 W W,Enchantment,Whenever a creature enters the battlefield und...,,,10E,rare,"{'commander': 'Legal', 'duel': 'Legal', 'legac...",True,"[4.0, 2.05, 5.94, 1.23]"
7,Angelic Wall,1 W,Creature — Wall,Defender (This creature can't attack.)\nFlying,0,4,10E,common,"{'commander': 'Legal', 'duel': 'Legal', 'legac...",True,"[0.05, 0.05, 0.01, 0]"
8,Angelic Wall,1 W,Creature — Wall,Defender (This creature can't attack.)\nFlying,0,4,10E,common,"{'commander': 'Legal', 'duel': 'Legal', 'legac...",True,"[1.6, 0, 0.24, 0.01]"
9,Aura of Silence,1 W W,Enchantment,Artifact and enchantment spells your opponents...,,,10E,uncommon,"{'commander': 'Legal', 'duel': 'Legal', 'legac...",True,"[20.0, 2.3, 30.9, 2.92]"


In [22]:
print(x[9])

1 W W Enchantment Artifact and enchantment spells your opponents cast cost {2} more to cast.
Sacrifice Aura of Silence: Destroy target artifact or enchantment.   10E uncommon {'commander': 'Legal', 'duel': 'Legal', 'legacy': 'Legal', 'modern': 'Legal', 'premodern': 'Legal', 'vintage': 'Legal'} True
